In [5]:
import os
import sys
import copy
import pickle
import numpy as np
import scipy as sci
import pandas as pd
from tqdm import tqdm
import matplotlib as mpl
from astropy import units 
import ChiantiPy.core as ch
from astropy.io import ascii
import matplotlib.pyplot as plt
from astropy.table import Table
from astropy.table import Column
import astropy.constants as const
from dynesty import utils as dyfunc
from dynesty.plotting import _quantile
from scipy.interpolate import interp1d
from dynesty import DynamicNestedSampler

from lmfit.models import Model
from specutils import Spectrum1D, SpectralRegion
from specutils.analysis import line_flux, equivalent_width
from astropy.nddata import StdDevUncertainty, NDUncertainty

distance = (9.72*units.pc).to(units.cm)
distanceErr = (0.004625*units.pc).to(units.cm)
radius   = (0.75*units.Rsun).to(units.cm)
radiusErr= (0.03*units.Rsun).to(units.cm)

%load_ext autoreload
%autoreload 2

sys.path.append('/Users/arcticfox/Documents/GitHub/cos_flares/src/')
from dem_modeling import ChiantiSetup, DEMModeling, setup_linelist
from utils import *
from cos_flares import FlaresWithCOS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## The Line Table Input

This table must have the following columns:
- Ion : ion name.
- wave_obs : observed center wavelength (Wavelength = center in vacuum).
- wmin : lower/bluer wavelength end of the line.
- wmax : upper/redder wavelength end of the line.
- quality : an array of 0s and 1s indicating whether to use that line or not.

In [2]:
line_table = Table.read('/Users/arcticfox/Documents/AUMic/aumic_linelist.csv',
                        format='csv', comment='#')

In [4]:
line_table[:3]

Ion,Wavelength,Questionable,Fluorescent,Forest,velocity_offset,wave_obs,vmin,vmax,wmin,wmax,quality
str7,float64,str1,str1,int64,float64,float64,float64,float64,float64,float64,float64
N II,1083.99,--,--,16,-2.6686968467985728,1083.9868159796877,-75.0,75.0,1083.7191686926449,1084.2688128929528,0.0
N II,1084.58,--,--,16,-2.6672495037960195,1084.5720700324375,-75.0,40.0,1084.3073844157816,1084.7220282861895,0.0
He II,1084.94,?,--,16,-5.332744743826398,1084.9186227666814,-75.0,75.0,1084.6545281212393,1085.2041723215475,0.0


## HST/COS specific setup to get wavelength & spectrum

In [ ]:
def load_data(path):
    wavelength, flux, err = load_binned_data(os.path.join(path,'interp_between_1.npy'))

    times = np.load(os.path.join(path,'times.npy'))
    orbits = np.load(os.path.join(path,'orbits.npy'))

    wavelength = np.full(flux.shape, wavelength)
    return wavelength, flux, err, times, orbits

w20, f20, e20, t20, o20 = load_data(path='/Users/arcticfox/Documents/AUMic/30s/reduced')
w30, f30, e30, t30, o30 = load_data(path='/Users/arcticfox/Documents/AUMic/30s/reduced2')

_, flare_orbits, flare_mask = np.load('/Users/arcticfox/Documents/AUMic/flare_mask2.npy',
                                      allow_pickle=True)

In [ ]:
fwc = FlaresWithCOS(wavelength=np.append(w20, w30, axis=0), 
                    flux=np.append(f20, f30, axis=0),
                    flux_err=np.append(e20, e30, axis=0), 
                    time=np.append(t20, t30)-t20[0], 
                    orbit=np.append(o20, o30+5))
fwc.load_line_table(path='/Users/arcticfox/Documents/GitHub/cos_flares/')

In [ ]:
mq = np.nansum(flare_mask,axis=0)==0
mean_quiescent = np.nanmean(fwc.flux[mq],axis=0)
mean_quiexcent_err = np.sqrt(np.nansum(fwc.flux_err[mq]**2, axis=0))/len(fwc.flux_err[mq])

## Creating the line list dictionary

In [ ]:
aumic_linelist = setup_linelist(fwc.wavelength[0], # wavelength array
                                mean_quiescent,    # spectrum
                                mean_quiexcent_err,# spectrum error
                                line_table,        # table of ions
                                distance, distanceErr, # distance & error
                                radius, radiusErr) # stellar radius & error

In [ ]:
pickle.dump(aumic_linelist, open('aumic_linelist.pkl','wb'))

## Setting up files from Chianti

In [ ]:
cs = ChiantiSetup(linelist=aumic_linelist,
                  logT_range=[4,8],
                  wave_range=[1,3200])

In [ ]:
cs.emissivity_functions()

In [ ]:
cs.get_all_ions(top=30)

In [ ]:
pickle.dump(cs.G_T, open('aumic_G_T.pkl','wb'))
pickle.dump(cs.linelist, open('aumic_linelist_updated.pkl', 'wb'))

## DEM Modeling

In [ ]:
lines = pd.read_pickle(r'aumic_linelist_updated.pkl')
#lines.pop('H2')
#lines.pop('SiI')
G_T = pd.read_pickle(r'aumic_G_T.pkl')

In [ ]:
dem = DEMModeling(linelist=lines,
                  G_T=cs.G_T,
                  logT_range=cs.logT_range)

In [ ]:
dem.create_DEM(quick_plot=True, specified_lines=list(dem.linelist.keys()))

In [ ]:
dem.estimate_EUV_from_DEM()

In [ ]:
pickle.dump(dem.linelist, open('aumic_dem_models.pkl','wb'))